# Lead Scoring Classification

In [1]:
# importing libraries
import pandas as pd
import numpy as np

In [2]:
#read csv
df = pd.read_csv("./course_lead_scoring.csv")

In [3]:
# most frequent industry
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
education        187
healthcare       187
technology       179
manufacturing    174
Name: count, dtype: int64

## fill null values
fill 'NA' for categorical columns and 0 for numerical columns

In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
columns = list(df.columns)
columns

['lead_source',
 'industry',
 'number_of_courses_viewed',
 'annual_income',
 'employment_status',
 'location',
 'interaction_count',
 'lead_score',
 'converted']

In [7]:
print(df['lead_score'].dtypes)

float64


In [8]:
for col in columns:
    if df[col].dtype=='float64' or df[col].dtype=='int64':
        df[col]=df[col].fillna(0.0)
    else:
        df[col]=df[col].fillna('NA')

## Correlation

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   object 
 1   industry                  1462 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1462 non-null   float64
 4   employment_status         1462 non-null   object 
 5   location                  1462 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [10]:
df[['interaction_count','number_of_courses_viewed']].corrwith(df['lead_score'])

interaction_count           0.009888
number_of_courses_viewed   -0.004879
dtype: float64

In [11]:
df[['number_of_courses_viewed','annual_income']].corrwith(df['interaction_count'])

number_of_courses_viewed   -0.023565
annual_income               0.027036
dtype: float64

## spliting of dataset

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_full_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
df_train,df_val = train_test_split(df_full_train,test_size=0.25,random_state=42)

In [14]:
df_full_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)
df_train.reset_index(drop=True,inplace=True)
df_val.reset_index(drop=True,inplace=True)

In [15]:
y_full_train = df_full_train['converted']
y_test = df_test['converted']
y_train = df_train['converted']
y_val = df_val['converted']

del df_full_train['converted']
del df_test['converted']
del df_train['converted']
del df_val['converted']

## mutual score

In [16]:
from sklearn.metrics import mutual_info_score

In [17]:
def mutual_score(series):
    return mutual_info_score(series,y_train)

In [18]:
categorical = list(df_train.dtypes[df_train.dtypes=='object'].index)

In [19]:
mi = df_train[categorical].apply(mutual_score)

In [20]:
sorted_mi = mi.sort_values(ascending=False)
sorted_mi.round(2)

lead_source          0.04
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

## featuring and training the model

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [22]:
dicts_train = df_train.to_dict(orient='records')
dicts_val = df_val.to_dict(orient='records')

In [23]:
dv = DictVectorizer(sparse = False)

In [24]:
X_train = dv.fit_transform(dicts_train)
X_val = dv.transform(dicts_val)

In [25]:
model = LogisticRegression(solver='liblinear',C=1,max_iter=1000,random_state=42)

In [26]:
model.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [31]:
y_pred = model.predict_proba(X_val)[:,1]

In [38]:
conversion_decision = (y_pred>=0.5)
conversion_decision
(y_val==conversion_decision).mean()

np.float64(0.6996587030716723)

In [52]:
def logistic_regression(df_train,df_test,y_train,y_test,c=1):
    dict_train = df_train.to_dict(orient='records')
    dict_test = df_test.to_dict(orient='records')
    dv = DictVectorizer(sparse = False)
    X_train = dv.fit_transform(dict_train)
    X_val = dv.transform(dict_test)
    model = LogisticRegression(solver='liblinear',C=c,max_iter=1000,random_state=42)
    model.fit(X_train,y_train)
    y_pred = model.predict_proba(X_val)[:,1]
    conversion_decision = (y_pred>=0.5)
    accuracy = (y_test==conversion_decision).mean()
    return accuracy 
    

In [53]:
logistic_regression(df_train,df_val,y_train,y_val)

np.float64(0.6996587030716723)

## feature elimination technique

In [94]:
original_accuracy = logistic_regression(df_train,df_val,y_train,y_val)
original_accuracy

np.float64(0.6996587030716723)

In [95]:
columns_to_check = ['industry','employment_status','lead_score']

In [96]:
columns = df_train.columns.to_list()

In [102]:

for i in columns_to_check:
    new_list = columns.copy()
    new_list.remove(i)
    df_1 = df_train[new_list]
    df_2 = df_val[new_list]
    accuracy = logistic_regression(df_1,df_2,y_train,y_val)
    print(f'accuracy after eliminating {i} is {accuracy}')
    print(f'original accuracy - {i} accuracy = {original_accuracy-accuracy}')
    print()

accuracy after eliminating industry is 0.6996587030716723
original accuracy - industry accuracy = 0.0

accuracy after eliminating employment_status is 0.6962457337883959
original accuracy - employment_status accuracy = 0.0034129692832763903

accuracy after eliminating lead_score is 0.7064846416382252
original accuracy - lead_score accuracy = -0.0068259385665528916



In [109]:
# best accuracy by elimating a feature
reg_strength_accuracy={}
reg_strength = [0.01, 0.1, 1, 10, 100]
for i in reg_strength:
    print(i)
    accuracy = logistic_regression(df_train,df_val,y_train,y_val,c=i)
    print(accuracy)
    reg_strength_accuracy[i]=accuracy
    


0.01
0.6996587030716723
0.1
0.6996587030716723
1
0.6996587030716723
10
0.6996587030716723
100
0.6996587030716723


In [110]:
reg_strength_accuracy

{0.01: np.float64(0.6996587030716723),
 0.1: np.float64(0.6996587030716723),
 1: np.float64(0.6996587030716723),
 10: np.float64(0.6996587030716723),
 100: np.float64(0.6996587030716723)}

In [114]:
sorted(reg_strength_accuracy.items(),key=lambda x: x[1])

[(0.01, np.float64(0.6996587030716723)),
 (0.1, np.float64(0.6996587030716723)),
 (1, np.float64(0.6996587030716723)),
 (10, np.float64(0.6996587030716723)),
 (100, np.float64(0.6996587030716723))]